In [1]:
# Preamble


import os
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline  

#os.chdir("/Users/macuser/Downloads/[ML]") 

In [ ]:
# Read Data
## Note: read_csv can automatically detect compressed file, and try to decompress it on the fly


df = pd.read_csv(dataname)   # for csv
# df = pd.read_excel(dataname, sheetname=page)   # for xls
# df = pd.read_stata(dataname)    # for dta

df.shape

In [ ]:
# Look at columns

df.columns.values

## Optional: replace column names

df.rename(index=str, columns={"Unnamed: 0": "ID"}, inplace = True)

In [ ]:
# Find duplicates and Missing Values

print('Number of duplicated observations:', df.duplicated().sum())
print('Number of missing values:', df['ref_domain_name'].isnull().sum())


## Optional: trim data based on duplicates and missing values
#df.drop_duplicates(inplace=True)
#df = df.dropna(['Column1'])

In [ ]:
# Detect common outliers due to self reporting:
## Age
print(sum(df.age > 122))  ## 122: the longest confirmed human lifespan record
print(sum(users.age < 18)) ## can vary, such as eligibility threshold

In [ ]:
# Describe Variables in Batch:

#describe non-numeric values
if (df.select_dtypes(include=['object']).shape[1]>0):
    des_obj = np.round(df.select_dtypes(include=['object']).describe(), 2).T
    print ('Object columns:',des_obj)
    
#Describe the rest
des_num = np.round(df.select_dtypes(exclude=['object']).describe(), 2).T
print ('Numeric columns:',des_num)

In [ ]:
# Snapshots of all kinds


## Sort:
df.sort(['A', 'B'], ascending=[1, 0])

## Count values:
df['A'].value_counts()
# df['A'].value_counts(sort=False)

## Visualized value counts:
df.gender.value_counts(dropna=False).plot(kind='bar', color='#FD5C64', rot=0)
plt.xlabel('Gender')

## Row Slicing:
df[101:105]
## Column slicing by number
df.iloc[:,2:4]
## Column slicing by label
df.loc[:,['ref_domain_name','domain_name']]

In [ ]:
# Numerical value manipulation


np.round(df['A'], 2)
df[log_A] = df[A].apply(lambda x: np.log(x+1))
df['A'].quantile([0.05, 0.1, 0.5, 0.9, 0.95])

bin = np.linspace(20,110,19)

In [ ]:
# Graphic representation


bin = np.linspace(0,12,13)
plt.hist(df['A'].dropna(),bins=bin)
plt.hist(df['A'].dropna(),bins=bin,log=True)

In [ ]:
# Relationship across variables


## Correlation:
df.iloc[:,[3,9,7]].corr()

## Summary statistics by group:
grouped = df.groupby(['A', 'B'])
df.groupby('A').groups
len(grouped)

grouped.get_group('amazon.com')['basket_tot']
grouped.get_group('amazon.com')['basket_tot'].sum()
grouped.get_group('amazon.com')['basket_tot'].mean()
grouped.get_group('amazon.com')['basket_tot'].std()
grouped.get_group('amazon.com')['basket_tot'].boxplot()



## apply function to each group (the function inside 'apply' can be user-defined function as well)
grouped['A'].apply(lambda x: x.describe())


## Cross tabulation (aggfunc example: np.sum, np.std, np.mean,...):
table1 = pd.crosstab(df.A, df.B, values=df.C, aggfunc=np.sum)

print(table1.to_string(na_rep=''))

In [ ]:
# Data Transformation


## get dummies and join it to the original df
dummies = pd.get_dummies(df['machine_id'], prefix='id')
df.join(dummies)

## standarlization/scaling

## replace missing value
df["col"].replace('-unknown-', np.nan, inplace=True)

## transform dtype (to, e.g. 'float', 'int', 'str', 'category') in batch:
categorical_features = [
    'affiliate_channel',
    'affiliate_provider',
    'country_destination'
]

for categorical_feature in categorical_features:
    df[categorical_feature] = df[categorical_feature].astype('category')
    
## special transformation: datetime
df['date_first_booking'] = pd.to_datetime(df['date_first_booking'])
df['date_first_active'] = pd.to_datetime((df.timestamp_first_active // 1000000), format='%Y%m%d')

In [ ]:
# Plotting with Seaborn

## Bar plot by group (gender in this case; can also do ex post grouping to continuous vars)
women = sum(users['gender'] == 'FEMALE')
men = sum(users['gender'] == 'MALE')

female_destinations = users.loc[users['gender'] == 'FEMALE', 'country_destination'].value_counts() / women * 100
male_destinations = users.loc[users['gender'] == 'MALE', 'country_destination'].value_counts() / men * 100

width = 0.4

male_destinations.plot(kind='bar', width=width, color='#4DD3C9', position=0, label='Male', rot=0)
female_destinations.plot(kind='bar', width=width, color='#FFA35D', position=1, label='Female', rot=0)

plt.legend()
plt.xlabel('Destination Country')
plt.ylabel('Percentage')

sns.despine()
plt.show()


## Histogram with density plot
sns.distplot(users.age.dropna(), color='#FD5C64')
plt.xlabel('Age')
sns.despine()

## Time series line plot
sns.set_style("whitegrid", {'axes.edgecolor': '0'})
sns.set_context("poster", font_scale=1.1)
users.date_account_created.value_counts().plot(kind='line', linewidth=1.2, color='#FD5C64')